In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *
from models import RobustPCANN

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

from pytorch_robust_pca import *

# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Tracking
from tqdm import trange

import sympy
import sympytorch

Running Python 3.9.7
You can use npar for np.array


In [2]:
# Loading the KS sol
DATA_PATH = "../deephpms_data/KS_simple3.pkl"
data = pickle_load(DATA_PATH)
t = data['t']
x = data['x']
X, T = np.meshgrid(x, t)
Exact = data['u'].T

# Adding noise
noise_intensity = 0.01
noisy_xt = True

Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
print("Perturbed Exact with intensity =", float(noise_intensity))

x_star = X.flatten()[:,None]
t_star = T.flatten()[:,None]
X_star = np.hstack((x_star, t_star))
u_star = Exact.T.flatten()[:,None]

if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

N = 5000
print(f"Fine-tuning with {N} samples")
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

# Convert to torch.tensor
X_u_train = to_tensor(X_u_train, True)
u_train = to_tensor(u_train, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names, base on the symbolic regression results
feature_names=('uf', 'u_x', 'u_xx', 'u_xxxx'); feature2index = {}

Loaded from ../deephpms_data/KS_simple3.pkl
Perturbed Exact with intensity = 0.01
Noisy (x, t)
Fine-tuning with 5000 samples


In [3]:
program = '''
-0.534833*u_xx-0.518928*u_xxxx-0.541081*uf*u_x
'''
pde_expr, variables = build_exp(program); print(pde_expr, variables)
mod = sympytorch.SymPyModule(expressions=[pde_expr]); mod.train()

-0.541081*u_x*uf - 0.534833*u_xx - 0.518928*u_xxxx {u_xx, u_x, uf, u_xxxx}


SymPyModule(expressions=(-0.541081*u_x*uf - 0.534833*u_xx - 0.518928*u_xxxx,))

In [4]:
class RobustPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None, pretrained=False, noiseless_mode=True, init_cs=(0.5, 0.5), init_betas=(0.0, 0.0)):
        super(RobustPINN, self).__init__()
        self.model = model
        if not pretrained: self.model.apply(self.xavier_init)
        
        self.noiseless_mode = noiseless_mode
        if self.noiseless_mode: print("No denoising")
        
        self.in_fft_nn = None; self.out_fft_nn = None
        self.inp_rpca = None; self.out_rpca = None
        if not self.noiseless_mode:
            # FFTNN
            self.in_fft_nn = FFTTh(c=init_cs[0])
            self.out_fft_nn = FFTTh(c=init_cs[1])

            # Robust Beta-PCA
            self.inp_rpca = RobustPCANN(beta=init_betas[0], is_beta_trainable=True, inp_dims=2, hidden_dims=32)
            self.out_rpca = RobustPCANN(beta=init_betas[1], is_beta_trainable=True, inp_dims=1, hidden_dims=32)
        
        self.callable_loss_fn = loss_fn        
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        self.diff_flag = diff_flag(self.index2features)
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, x, t):
        H = torch.cat([x, t], dim=1)
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, X_input, X_input_noise, y_input, y_input_noise, update_network_params=True, update_pde_params=True):
        # Denoising process
        if not self.noiseless_mode:
            # (1) Denoising FFT on (x, t)
            # This line returns the approx. recon.
            X_input_noise = cat(torch.fft.ifft(self.in_fft_nn(X_input_noise[1])*X_input_noise[0]).real.reshape(-1, 1), 
                                torch.fft.ifft(self.in_fft_nn(X_input_noise[3])*X_input_noise[2]).real.reshape(-1, 1))
            X_input_noise = X_input-X_input_noise
            X_input = self.inp_rpca(X_input, X_input_noise, normalize=True)
            
            # (2) Denoising FFT on y_input
            y_input_noise = y_input-torch.fft.ifft(self.out_fft_nn(y_input_noise[1])*y_input_noise[0]).real.reshape(-1, 1)
            y_input = self.out_rpca(y_input, y_input_noise, normalize=True)
        
        grads_dict, u_t = self.grads_dict(X_input[:, 0:1], X_input[:, 1:2])
        
        total_loss = []
        # MSE Loss
        if update_network_params:
            mse_loss = F.mse_loss(grads_dict["uf"], y_input)
            total_loss.append(mse_loss)
            
        # PDE Loss
        if update_pde_params:
            u_t_pred = self.callable_loss_fn(**grads_dict)
            l_eq = F.mse_loss(u_t_pred.squeeze(-1), u_t)
            total_loss.append(l_eq)
            
        return total_loss
    
    def grads_dict(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        derivatives = group_diff(uf, (x, t), self.diff_flag[1], function_notation="u", gd_init={"uf":uf})
        
        return derivatives, u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    def neural_net_scale(self, inp): 
        return -1.0+2.0*(inp-self.lb)/(self.ub-self.lb)

In [5]:
model = TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1], activation_function=nn.Tanh, bn=nn.LayerNorm, dropout=None)

# Pretrained model
semisup_model_state_dict = torch.load("./saved_path_inverse_small_KS/simple3_semisup_model_state_dict_250labeledsamples250unlabeledsamples.pth")
parameters = OrderedDict()
# Filter only the parts that I care about renaming (to be similar to what defined in TorchMLP).
inner_part = "network.model."
for p in semisup_model_state_dict:
    if inner_part in p:
        parameters[p.replace(inner_part, "")] = semisup_model_state_dict[p]
model.load_state_dict(parameters)

pinn = RobustPINN(model=model, loss_fn=mod, 
                  index2features=feature_names, scale=True, lb=lb, ub=ub, 
                  pretrained=True, noiseless_mode=False)

Using old implementation of TorchMLP. See models.py for more new model-related source code.


In [6]:
_, x_fft, x_PSD = fft1d_denoise(X_u_train[:, 0:1], c=-5, return_real=True)
_, t_fft, t_PSD = fft1d_denoise(X_u_train[:, 1:2], c=-5, return_real=True)
_, u_train_fft, u_train_PSD = fft1d_denoise(u_train, c=-5, return_real=True)

In [7]:
x_fft, x_PSD = x_fft.detach(), x_PSD.detach()
t_fft, t_PSD = t_fft.detach(), t_PSD.detach()

In [8]:
pinn = load_weights(pinn, "./saved_path_inverse_small_KS/noisy2_final_finetuned_pinn_5000.pth")

Loaded the model's weights with strict=False


In [9]:
def closure():
    if torch.is_grad_enabled():
        optimizer2.zero_grad()
    losses = pinn.loss(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), u_train, (u_train_fft, u_train_PSD), update_network_params=True, update_pde_params=True)
    l = sum(losses)
    if l.requires_grad:
        l.backward(retain_graph=True)
    return l

def mtl_closure():
    losses = pinn.loss(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), u_train, (u_train_fft, u_train_PSD), update_network_params=True, update_pde_params=True)
    updated_grads = []
    
    for i in range(len(losses)):
        optimizer1.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()): 
        param.grad = updated_grads[0][idx]+updated_grads[1][idx]
        
    return sum(losses)

In [10]:
epochs1, epochs2 = 200, 0
# TODO: Save best state dict and training for more epochs.
optimizer1 = MADGRAD(pinn.parameters(), lr=1e-7, momentum=0.9)
pinn.train(); best_train_loss = 1e6

print('1st Phase optimization using Adam with PCGrad gradient modification')
for i in range(epochs1):
    optimizer1.step(mtl_closure)
    if (i % 10) == 0 or i == epochs1-1:
        l = mtl_closure()
        print("Epoch {}: ".format(i), l.item())
        print([x.item() for x in pinn.callable_loss_fn.parameters()])
        
optimizer2 = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(500*1.25), history_size=300, line_search_fn='strong_wolfe')
print('2nd Phase optimization using LBFGS')
for i in range(epochs2):
    optimizer2.step(closure)
    if (i % 10) == 0 or i == epochs2-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())

1st Phase optimization using Adam with PCGrad gradient modification
Epoch 0:  0.008657058700919151
[-1.0003485679626465, -0.9991112947463989, -0.9980955123901367]
Epoch 10:  0.006967166438698769
[-1.0001983642578125, -0.9992867112159729, -0.9979602694511414]
Epoch 20:  0.006393918767571449
[-0.9999387860298157, -0.9995732307434082, -0.9977527856826782]
Epoch 30:  0.006182690151035786
[-0.9996455311775208, -0.9998957514762878, -0.9975160956382751]
Epoch 40:  0.0060951923951506615
[-0.9993488788604736, -1.00022292137146, -0.9972723126411438]
Epoch 50:  0.006088713649660349
[-0.9990496635437012, -1.0005509853363037, -0.9970305562019348]
Epoch 60:  0.006117525976151228
[-0.9987487196922302, -1.0008784532546997, -0.9967945218086243]
Epoch 70:  0.0061643775552511215
[-0.9984502196311951, -1.0012025833129883, -0.9965620040893555]
Epoch 80:  0.006208677776157856
[-0.9981558918952942, -1.0015230178833008, -0.9963310360908508]
Epoch 90:  0.006255767773836851
[-0.9978655576705933, -1.001839518547

In [11]:
pred_params = [x.item() for x in pinn.callable_loss_fn.parameters()]
print(pred_params)

[-0.9947568774223328, -1.0052711963653564, -0.9935149550437927]


In [12]:
errs = 100*np.abs(np.array(pred_params)+1)
print(errs.mean(), errs.std())

0.5666454633076986 0.05789442242742208


In [13]:
### New results ###
# w/o DFT

# clean
# [-0.9948897361755371, -1.0049610137939453, -0.9951441287994385]
# 0.4975716272989909 0.01043744028051521

# noisy labels
# [-0.9948846697807312, -1.0065652132034302, -0.9950999617576599]
# 0.5526860555013021 0.07394681988603513

# noisy (x, t) and labels
# [-0.9892916083335876, -1.011358618736267, -0.9886858463287354]
# 1.1127054691314697 0.029659549537678777

# noisy (x, t) and labels | /w DFT
# [-0.9947568774223328, -1.0052711963653564, -0.9935149550437927]
# 0.5666454633076986 0.05789442242742208

In [14]:
### Without AutoEncoder ###

# Clean Exact and (x, t)
# [-0.999634325504303, -0.9994997382164001, -0.9995566010475159]
# (0.04364450772603353, 0.005516461306387781)

# Pretrained with final_finetuned_pinn_5000 (do not use / report)
# [-0.9996052980422974, -0.9995099902153015, -0.9995319247245789]
# 0.04509290059407552 0.0040754479422416435

# Noisy Exact and clean (x, t)
# [-0.9967969655990601, -0.9969800114631653, -0.9973703026771545]
# (0.2950906753540039, 0.023910676954986623)

# Noisy Exact and noisy (x, t)
# [-0.9975059032440186, -0.9962050914764404, -0.9969104528427124]
# 0.3126184145609538 0.05316856937153804

In [15]:
### Without AutoEncoder & With Robust PCA ###

# Noisy Exact and clean (x, t)
# [-1.0003160238265991, -1.0005097389221191, -0.9997726678848267]
# (0.035103162129720054, 0.011791963483533422)

# Noisy Exact and noisy (x, t) + Robust PCA
# [-1.000351071357727, -0.9991073608398438, -0.9980993866920471]
# (0.08140603701273601 0.09209241474722876)

In [16]:
### Without AutoEncoder & With Double Beta-Robust PCA ###

# Noisy Exact and clean (x, t)
# [-1.0013889074325562, -0.998389720916748, -1.0010501146316528]
# 0.1349767049153646 0.023035484282371465

# Noisy Exact and noisy (x, t) + Robust PCA
# [-0.9980248212814331, -0.9982557892799377, -0.998869001865387]
# 0.1616795857747396 0.03562172791384541